In [1]:
import os                                                                          
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"                                       
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import tensorflow as tf
session_config=tf.ConfigProto(
        gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.1, allow_growth=True), allow_soft_placement=True, log_device_placement=False)
sess = tf.Session(config=session_config)

from tensorflow.contrib.slim.nets import resnet_v2, resnet_utils
import tensorflow as tf
from tensorflow.contrib import layers as layers_lib
from tensorflow.python.ops import variable_scope
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.contrib import slim
from tensorflow.nn import ctc_loss, conv2d
import numpy as np
resnet_v2_block = resnet_v2.resnet_v2_block
resnet_v2 = resnet_v2.resnet_v2

import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import ipdb

In [3]:
def resnet_v2_26_base(inputs,
                 num_classes=None,
                 is_training=True, # True - due to update batchnorm layers
                 global_pool=False,
                 output_stride=1, # effective stride 
                 reuse=None,
                 include_root_block=False, #first conv layer. Removed due to max pool supression. We need large receprive field
                 scope='resnet_v2_26'):
  
    """
    Tensorflow resnet_v2 use only bottleneck blocks (consist of 3 layers).
    Thus, this resnet layer model consist of 26 layers.
    I put stride = 2 on each block due to increase receptive field.

    """
    blocks = [
      resnet_v2_block('block1', base_depth=64, num_units=2, stride=2),
      resnet_v2_block('block2', base_depth=128, num_units=2, stride=2),
      resnet_v2_block('block3', base_depth=256, num_units=2, stride=2),
      resnet_v2_block('block4', base_depth=512, num_units=2, stride=2),
    ]
    return resnet_v2(
      inputs,
      blocks,
      num_classes,
      is_training,
      global_pool,
      output_stride,
      include_root_block,
      reuse=reuse,
      scope=scope)

def make_ocr_net(inputs, num_classes, is_training=True):
    '''
    Creates neural network graph.
    Image width halved and it's define timestamps width (feature sequence length) 
    No activation after output (no softmax), due to it's presence at ctc_loss() and beam_search().
    After resnet head features are resized to be [batch,1,width,channel], and after that goes 1x1 conv 
    to make anology of dense connaction for each timestamp.
    
    input: batch of images
    output: tensor of size [batch, time_stamps_width, num_classes]
    '''
    with tf.variable_scope('resnet_base', values=[inputs]) as sc:
        with slim.arg_scope([slim.conv2d],
                              activation_fn=None, normalizer_fn=None):
            net = resnet_utils.conv2d_same(inputs, 64, 7, stride=2, scope='conv1') #root conv for resnet
            #net = slim.max_pool2d(net, [3, 3], stride=2, scope='pool1') # due to enlarge of receptive field
            net = resnet_v2_26_base(net, output_stride=1, is_training = is_training)[0] # ouput is a tuple of last tensor and all tensors 
    with tf.variable_scope('class_head', values=[net]) as sc:
        net = tf.transpose(net, [0,3,1,2]) # next 4 lines due to column to channel reshape. [batch,c,h,w]
        _,c,h,_ = net.get_shape() # depth of input to conv op tensor should be static (defined)
        shape = tf.shape(net)
        net = tf.reshape(net, [shape[0], c*h, 1, shape[3]])
        net = tf.transpose(net,[0,2,3,1]) # back to [batch,h,w,c] = [batch,1,w,features*h]
        net = layers_lib.conv2d(net, num_classes, [1, 1], activation_fn=None) #CTC got softmax [batch,1,w,num_classes]
        net = tf.squeeze(net,1) #[batch,w,num_classes]
        return net

def get_prediction(output_net, seq_len, merge_repeated=False):
    '''
    predict by using beam search
    input: output_net - logits (without softmax) of net
           seq_len - length of predicted sequence 
    '''
    net = tf.transpose(output_net, [1, 0, 2]) #transpose to [time, batch, logits]
    
#     decoded, prob = tf.nn.ctc_beam_search_decoder(net, seq_len, merge_repeated=merge_repeated)
#     return decoded, prob
    decoded = [tf.contrib.layers.dense_to_sparse(tf.argmax(net, -1))]
    prob = tf.ones(tf.shape(output_net)[0], 1) #batch size ,top_paths
    return decoded, prob


In [4]:
with open('../../data_generator/valid_charset.txt', 'r', encoding = 'utf-8') as f:
    valid_charset = f.read()
all_chars = list(valid_charset)
char_to_indx = dict(zip(all_chars,range(len(all_chars))))
num_classes = len(all_chars)

In [5]:
print(dict(zip(range(len(all_chars)), all_chars)))

{0: 'Е', 1: 'Щ', 2: 'Э', 3: 'e', 4: 'M', 5: 'S', 6: 'н', 7: 'ь', 8: 'В', 9: 'у', 10: 'I', 11: 'R', 12: '7', 13: '8', 14: '"', 15: 'й', 16: 'Ф', 17: 'u', 18: 'B', 19: 'П', 20: 'Я', 21: '=', 22: 'Ж', 23: '-', 24: 'и', 25: '№', 26: 'y', 27: 'n', 28: 'T', 29: '!', 30: 'a', 31: 'З', 32: 'ъ', 33: 'X', 34: 'O', 35: 'Ю', 36: 't', 37: 'Ч', 38: 'п', 39: '+', 40: 'v', 41: 'Г', 42: 'f', 43: 'z', 44: '&', 45: 'F', 46: 'з', 47: '.', 48: 'Б', 49: ')', 50: '3', 51: 'Y', 52: 'х', 53: '6', 54: '*', 55: 'Ё', 56: '_', 57: 'N', 58: '9', 59: 'q', 60: 'r', 61: 'K', 62: 'd', 63: '0', 64: '%', 65: 'U', 66: ',', 67: 'c', 68: 'W', 69: 'У', 70: 'р', 71: 'j', 72: 'E', 73: 'к', 74: 'g', 75: 'ш', 76: 'ж', 77: '4', 78: '1', 79: '~', 80: 'D', 81: 'Х', 82: '5', 83: 'Н', 84: 'G', 85: 'С', 86: 'Z', 87: 'k', 88: 'Л', 89: 'ё', 90: 'J', 91: 'ф', 92: 'w', 93: 'ю', 94: 'д', 95: 'л', 96: 'К', 97: 'т', 98: 'A', 99: 'Q', 100: 'м', 101: ':', 102: 'H', 103: 'ы', 104: 'е', 105: 'с', 106: 'г', 107: '(', 108: 'l', 109: 'Т', 110: '2',

In [6]:
numeric_chars = '0123456789,.'

In [7]:
class OCRModel(object):
    def __init__(self, charset, input_image_batch, sequence_labels, is_training=True, learning_rate=1e-4,
                decay_steps=2**16, decay_rate=0.9, decay_staircase=False, momentum=0.9):
        self.charset = charset
        self.num_classes = len(charset)+2 #indexing starts from one, +empty char. Yes, I know that I got 0-th rudimental output, whatever
        self.is_training = is_training
        self.learning_rate = learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate 
        self.decay_staircase = decay_staircase
        self.momentum = momentum
        mapping_char = tf.constant(list(charset))
        self.table = tf.contrib.lookup.index_to_string_table_from_tensor(
                    mapping_char, default_value="")

        self.build_for_prediction(input_image_batch)
        
    def build_for_prediction(self, input_image_batch):
        
        self.input_image_batch = input_image_batch
        net = make_ocr_net(self.input_image_batch, self.num_classes, is_training=self.is_training)
        self.net = net
        self.prediction = get_prediction(output_net=net, 
                                         seq_len=[tf.shape(self.net)[1]], merge_repeated=False) # tuple(decoded, prob). decoded - list of top paths. I use top1
        pred_dense = tf.sparse_to_dense(self.prediction[0][0].indices, self.prediction[0][0].dense_shape, 
                                               self.prediction[0][0].values)
#         pred_dense = self.prediction
        self.pred_dense = pred_dense
        self.prediction_string = tf.reduce_join(self.table.lookup(tf.cast(pred_dense, tf.int64)-1))
        

In [8]:
import json
from skimage import io
from skimage.transform import rescale
from skimage.color import rgb2gray
def resize_height(image, max_dim):
    h, w = image.shape
    scale_factor = max_dim/h
    output_image = rescale(image.copy(), scale = scale_factor, order = 3)
    return output_image

In [9]:
graph = tf.Graph()
with graph.as_default():

    features = tf.placeholder(tf.float32, shape = [None, 32, None, 1])
    model = OCRModel(charset = all_chars, input_image_batch=features, 
                     sequence_labels=None, is_training=False)
    init = tf.global_variables_initializer()
    table_init = tf.tables_initializer()
    saver = tf.train.Saver()

In [10]:
from lev_dist import distance

In [11]:
def is_num_string(string):
    return len(set(string).intersection(set(numeric_chars))) == len(set(string))

In [12]:
LOG_DIR = 'log/'
with tf.Session(graph=graph) as sess:
#     run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
#     run_metadata = tf.RunMetadata()
    sess.run([init, table_init])
    try:
        saver.restore(sess, LOG_DIR + "model_lev_dist_0.2719941635845625.ckpt")
    except:
        print('cant restore')
    with open('../datasets/labeled_strings.json', 'r') as f:
        string_data = json.load(f)
    match = 0
    lev_dist = 0
    
    for example in string_data:
        image = io.imread('../datasets/' + example['path'])
        image = rgb2gray(image)
        image = resize_height(image, 32)
#         image = image/255 #as image scaled to [0,1]
        image = image - 0.5
        string = example['context']
        pred_string = sess.run([model.prediction_string], feed_dict={features: image[np.newaxis,...,np.newaxis]})
        pred_string = pred_string[0].decode('utf-8')
        print(pred_string.upper(), string)
        lev_dist += distance(pred_string.upper(), string) 
        if pred_string.upper() == string:
            match+=1


INFO:tensorflow:Restoring parameters from log/model_lev_dist_0.2719941635845625.ckpt


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


ШWИМ №М РИС ДЛИННОЗЕРНЫЙ 800Г ЯРМАРКА
DAT  :П=0=ФK Data : 11-08-2017
РУБ РУБ
89 89
78 78
11  КГ=1112,23PYБ 1 КГ = 112,23РУБ
PИИC БЕЛЬИ ДЛИИНО3ЕPHЫИ 9 рис белый длинозерный 900г
MИСTРAЛЬЬ  OРИEHТ МИСТРАЛЬ ОРИЕНТ
DNT~Q2O DATA :27-10-2017
4KГ=Я9 R9NVБ 1 кг = 82.89 руб
PУБ РУБ
60| 60
74 74
PИC 3OЛOTИСТЫИ  9ЮГ AHГСTPEM РИС ЗОЛОТИСТЫЙ 900Г АНГСТРЕМ
DАTА  :ЭO-08~2OL7 DATA :30-08-2017
 KГ=1  777NVБ 1 КГ =100.77 РУБ
РУБ РУБ
669 69
90 90
PУБ РУБ
333 33
61 61
1 КГ"*48.90РУП 1 КГ =45.90РУБ
РУБ РУБ
90 90
445 45
/CДWИJPHЫЫИ А3А1СXИ РИС ДЛИННОЗЕРНЫЙ АЗИАТСКИЙ
ПАСCИM ПАССИМ
ATЯ \№0==10=H DATA : 30-10-2017
11 КГ=1115,88РУБ 1 КГ =115,88РУБ
РУБ РУБ
94 94
57 57
1КГ=63 90NVБ 1 КГ =63.90 РУБ
PУБ РУБ
90| 90
63 63
1 KГ=77,80PY0 1 КГ =77,80РУБ
РУБ| РУБ
90 90
38 38
1 КГ=153,ЗJPУБ 1 КГ =153,33РУБ
РУБ| РУБ
33| 33
6 61
1 KГ=77,80PУБ 1 КГ =77,80РУБ
PУБ РУБ
90 90
38 38
1 КГ=1553,33PУБ 1 КГ =153,33РУБ
PУБ РУБ
33| 33
61 61
СУПБOБO8ЫИ 25Г ЯPMPK СУП БОБОВЫЙ 250Г ЯРМАРКА
DATА  :11~1O-2O17 DATA :11-10-2017
222 122
00) 00


3 37
11  KГ  =775..80PУБ 1 КГ =75.80РУБ
PУБ РУБ
90| 90
3 37
11  KГ==1130,,З4PYБ 1 КГ =130,34РУБ
РУБ РУБ
17 1 7
65 65
PCПPОПAPEУRЫИ1KГ KД РИС ПРОПАРЕННЫЙ 1 КГ КД
РYБ РУБ
990 90
556 56
11КГ=75.80PУБ 1 КГ =75.80 РУБ
PYБ РУБ
90| 90
3 37
11 KГ=130,34PУБ 1 КГ =130,34РУБ
PУБ РУБ
117 1 7
65 65
11KГГ=556 S0РУУБ 1 КГ=56.90РУБ
PУБ РУБ
90 90
56 56
DATВ   :1L1~OB-2O17 DATA :11-08-1017
1KГ=422 23NVБ 1 КГ =112.23РУБ
РУБ РУБ
78 78
89 89
PИС  ДЛЯ  ПЛOВА  900ГР РИС ДЛЯ ПЛОВА 900ГР
DDATTА  ::O4-O2-2O177 DATA :04-02-2017
1 КГ==911,29РУБ 1 КГ =91,29РУБ
РУБ РУБ
16| 16
82 82
СУУПБОБОВЫИ 25Г  ЯPPМAPKА СУП БОБОВЫЙ 250Г ЯРМАРКА
2 122
00 00
PУБ РУБ
  KГ=4488,00PYБ 1 КГ =488,00РУБ
DАTA :1L-1O-2O177 DATA :11-10-2017
PИС  5X100ГР ДOHГГУСTО  МDD| РИС 5X100 ДОН ГУСТО MDD
DАTА :199-09-2D17 DATA :19-09-2017
1 KГ =75,80PYБ 1 КГ=75,80РУБ 
РУБ РУБ
90| 90
37 37
РИС   ДЛЯ  ПЛOBА  900ГР РИС ДЛЯ ПЛОВА 900ГР
DATAO2~№ DATA :04-02-2017
11 КГ ==911,,29РУБ 1 КГ =91,29РУБ
РYБ РУБ
16 1 6
822 82
PИИС  5Х100ГP  ДOHГУСTО  МDO РИС 5Х100

РУБ РУБ
52 52
996 96
PИС  КУБАHСКИИ 9OOГP РИС КУБАНСКИЙ 900ГР
DATА  :28-IO-2OI77 DATA :28-10-2017
11 КГ =79,77PУБ 1 КГ =79,77РУБ
PУБ РУБ
79 79
7 7 1
PЮCЖ09AЖДP РИС КЛАССИК 0,9КГ КРАСНОДАР
DATA  :28-O1~20I7 DATA :28-01-2017
11_Г =777З8PYБ 1 КГ =77,38РУБ
РУБ РУБ
64 64
69 69
P"ЗРАЮБАHX3HЫ РИС "ЭКСТРА" КУБАНСКИЙ ЭЛИТНЫЙ
900Г 900Г
DATА :O4-O0-2OL7 DATA :04-08-2017
==88DVБ 1 КГ =89,78РУБ
РУC РУБ
80 80
80 80
PИС KУБАHСKИИИ 9O0ГР РИС КУБАНСКИИ 900ГР
DАTTА  ::28-LO-2O17 DATA :28-10-2017
11 KГ =79,777PУБ 1 КГ =79,77РУБ
РУБ РУБ
79 79
1 7 1
PЫИДЖПCT0 1MO РИС КРУГЛЫЙ ДОН ГУСТО 1КГ MDD
DATA  :2O-1O-2O177 DATA :20-10-2017
11КГ =56,,90PУБ 1 КГ =56,90РУБ
PУБ РУБ
90 90
556 56
ПACСИM ПАССИМ
1KГ=115,BBРУБ| 1 КГ =115,88РУБ
РУБ РУБ
944 94
5 57
PЮУГПЫИИПРVMЮRСXИ5 РИС КРУГЛЫЙ ПРИМОРСКИЙ 500Г
ПАССИM ПАССИМ
DАTA :22-1O--2OI7 DATA :22-10-2017
КГ==115,88PУБ 1 КГ =115,88РУБ
PУБ РУБ
94 94
57 57
  KГ=556,90РУБ 1 КГ = 56,90РУБ
PУБ РУБ
90| 90
56 56
PИС KPУГЛЫЫИ ПРИМOРСKИИ 5ЮГГ РИС КРУГЛЫЙ ПРИМОРСКИЙ 500Г
ПAССИМ ПАССИМ

...

100  ГР=106" 100 ГР = 106 
1KГ==  679 1 КГ = 679
PУБ. РУБ.
PУБ. РУБ.
РУБ. РУБ.
РУБ. РУБ.
PУБ. РУБ.
PУБ.. РУБ.
1KГ =1319" 1 КГ =1319
PУБ. РУБ.
РУБ.. РУБ.
РУБ. РУБ.
1OOГР=446 100ГР = 46
1KГ==1049" 1 КГ = 1049
РУБ.. РУБ.
100ГР=3229" 100 ГР = 329
PУБ.. РУБ.
100ГP = 22455" 100 ГР = 245 
100ГPP = 232" 100 ГР = 232
11KГ=999* 1 КГ = 99995
1O0ГР=329" 100 ГР = 32940
PУБ. РУБ.
PУБ. РУБ.
РУБ.. РУБ.
PУБ. РУБ.
РУБ. РУБ.
100 ГР=329 100 ГР = 32940
1KГ==499" 1 КГ = 49997
PYБ. РУБ.
1D0 ГР = 1177 100 ГР = 117 
РУБ. РУБ.
РУБ. РУБ.
100 ГР=147* 100 ГР = 14748
100ГР== 388" 100 ГР = 38888
1KГ== 649 1 КГ = 64950
1O0  ГР= 128 100 ГР = 12800
100ГР =  40 100 ГР = 4000
100 ГР=92" 100 ГР = 92
1NN ГD1QQ 100 ГР = 199
1КГ~ 229 1 КГ = 224975
100ГР -239" 100 ГР = 239
Э100 ГP=93" 100 ГР = 93
1100 ГР=  83" 100 ГР = 83
00 ГP  139* 100 ГР = 139
РYБ. РУБ.
-2Г" 100ГР = 242
РУБ.. РУБ.
РYБ. РУБ.
РУБ. РУБ.
РУБ РУБ.
--19% -19%
РYБ. РУБ.
PУБ. РУБ.
PУБ. РУБ.
PYБ. РУБ.
PУБ. РУБ.
1KГ=4411" 1 КГ =411
-17%% -17%
1KГ=7734* 1 КГ = 73495


РYБ. РУБ.
1OO ГP=69..99 100 ГР =69.99
OГР=57.99 100 ГР = 57.99
РYБ. РУБ.
РУБ. РУБ.
РУБ. РУБ.
1KГ==259" 1КГ = 259
PУБ. РУБ.
PУБ. РУБ.
1KГ= 157" 1 КГ =157
YБ. РУБ.
РУБ. РУБ.
PУБ. РУБ.
1KГ==300" 1 КГ = 300
PУБ. РУБ.
PYБ РУБ
11KГ=255" 1 КГ =255
1KГ==228" 1 КГ = 228
1KГ= 1577" 1КГ =157
РУБ. РУБ.
PYБ. РУБ.
РУБ.. РУБ.
PУУБ. РУБ.
РУБ. РУБ.
PУБ РУБ.
D0 ГPP + 99" 100 ГР = 99
PУБ. РУБ.
10OР= 89" 100ГР = 8999
*PУБ.. РУБ.
PУБ. РУБ.
PУБ. РУБ.
РУБ. РУБ.
OШТ=91 100 ШТ = 91
PУБ. РУБ.
РУБ РУБ.
PYБ. РУБ.
PУБ. РУБ.
PУБ. РУБ.
PУБ. РУБ.
JOO ГP=60 100 ГР = 6029
РУБ.. РУБ.
1OOГP=З9.55O 100ГР =39.50
PУБ. РУБ.
PУБ. РУБ.
)OO Г* 128.32 100 ГР = 128.32
LOO  ГP==93.50 100 ГР = 93.50
PУБ. РУБ
 R-5O.65 100 ГР = 50.65
PУБ. РУБ.
10OГP=44S.З5 100 ГР = 45.35
PУБ. РУБ.
1100ГP =149* 100 ГР =14999
JOORP=76" 100 ГР = 76
РУ6. РУБ.
1OOГP=98" 100 ГР = 98
1OP-12Z" 100 ГР = 127
АKЦИЯ АКЦИЯ
PУБ. РУБ.
PУББ. РУБ.
1OORP=69" 100ГР = 69%
1OO ГP=1OO.777 100 ГР = 100.77
O-2З" 100 ГР = 23
1OORP=17" 100ГР = 1733
РYБ. РУБ.
PУБ. РУБ.
PУБ. РУ

1OOГP=59.99 100 ГР = 59.99
10O ГP=178" 100 ГР = 17899
1OГP.= LL78" 100 ГР = 17899
1OOГP=L445"" 100 ГР = 145
1OO ГP==2399.99 100 ГР = 239.99
100  ГР=236* 100 ГР = 23699
PУБ.. РУБ.
100 ГP= 107 100 ГР = 10713
1OO ГP==186..99 100 ГР = 186.99
1OО P =92.OО 100 ГР = 92.00
1OO  ГP==81" 100 ГР = 8199
10O ГP=677 100 ГР = 6705
100ГP= 57" 100 ГР = 57
1KГ== 366" 1 КГ = 36694
1OOГР=92" 100 ГР = 9221
1OOГP==83" 100 ГР = 8350
1OГP=114.99 100 ГР = 114.99
1OOГP=1O4..99 100 ГР = 104.99
1ОORP=9O.55 100 ГР = 90.55
РYБ. РУБ.
JOO ГP=61 100 ГР = 6110
1O0ГP=661 100 ГР = 6110
PУБ. РУБ.
1OO ГP==9O.OO 100 ГР = 90.00
110 ГР - 1109* 100 ГР = 109
PУБ. РУБ.
1OO RP=299"" 100 ГР = 29998
10O ГP=69"" 100 ГР = 6999
1OOГP=18" 100 ГР = 1811
1OOГP=2O.38 100Г ГР = 20.38
РYБ. РУБ.
РУ6 РУБ
1OORP=35.З44 100 ГР = 35.34
1OOГP=35" 100 ГР = 3557
1OO ГP==З1" 100 ГР = 3311
10OГP = З0" 100 ГР = 30
РУББ. РУБ.
РYБ. РУБ.
OOP-148" 100 ГР = 14842
10OГP=89" 100 ГР = 89
РУБ. РУБ.
PУБ. РУБ.
1OO ГP=19" 100 ГР = 19
1OO ГP==37.99 100 ГР = 37.99
1

1OO ГP=88" 100 ГР = 88
1KГ=2249* 1 КГ = 224992
1KГ==86 1 КГ = 8665
РУБ. РУБ.
1KГ =488" 1 КГ = 488
1 KГ =  1422 1 КГ = 1422
1Л==1139* 1 Л = 13945
1KГ -87 1 КГ = 8700
РУБ. РУБ.
РУБ. РУБ.
11 KГ=1162 1 КГ = 116238
1OO RP=64 100 ГР = 64
1Д= I65" 1 Л = 165
1KГ=79 1 КГ = 7978
100ГР= 22 100 ГР = 22
10OГP=27 100 ГР = 2701
1KГ=2231 1 КГ = 23155
1KГ==296* 1 КГ = 296
1КГ==196 1 КГ = 19684
1KГ=2O44" 1 КГ = 204
1OOГР==211* 100 ГР = 21
1KГ=203 1 КГ = 203
РУБ. РУБ.
РУБ. РУБ.
1KГ= 89" 1 КГ = 89
1KKГ=295" 1 КГ = 295
PУБ. РУБ.
1KГ=134" 1 КГ = 134
11KГ==2289 1 КГ = 289
11KГ=216** 1 КГ = 21695
РУБ. РУБ.
1KГ=123* 1 КГ = 123
PУБ. РУБ.
РУБ. РУБ.
1KГГ= 1399" 1 КГ = 1399
100 ГP=774" 100 ГР = 74
PУ6. РУБ.
11KГ = 2222 1 КГ = 222
1KГ== 222 1 КГ = 222
1КГГ =381" 1 КГ = 38129
1KГ=  366" 1 КГ = 36694
1KГ= 22" 1 КГ = 227
1KГ==242 1 КГ = 242
1КГ==  1105 1 КГ = 105
1KГ= 11262" 1 КГ = 1262
1KГ= 2177* 1 КГ = 21745
PУБ.. РУБ.
PУБ. РУБ.
РУБ. РУБ.
РУББ. РУБ.
11KГ=222 1КГ = 22290
1Л= 1 Л = 141
PУБ.. РУБ.
1KГ=1104" 1 КГ = 104


1ЙORP=61" 100 ГР = 61
PУБ.. РУБ.
OOГP=31" 100 ГР = 3111
PУБ. РУБ.
PУБ. РУБ.
1KГ== 879 1 КГ = 87996
РУБ. РУБ.
100 ГР = 130 100 ГР = 13020
ЧГР-299" 100 ГР = 299
100ГР = 130" 100 ГР = 130
1O0 ГР=7O 100 ГР = 7000
OГР= 130 100 ГР = 13020
0ГP* 15 100 ГР = 115
11KГ=  1266" 1 КГ = 1266
1OO ГР=73 100 ГР = 73
-11O% -10%
 100 ГР = 166
100ГР= 1O6" 100 ГР =10666
11KГ=259" 1 КГ = 259
11KГ=255" 1 КГ =25590
1KГ= 127" 1 КГ = 12745
D100  ГP=111 100 ГР = 11160
10OГР=  135" 100 ГР = 135
OГР ~  1O5" 100 ГР = 105
100ГР=  120 100 ГР = 12000
1OОГP = 72" 100 ГР = 7217
100 ГP=69 100 ГР = 6956
100ГГР  = 1142" 100 ГР = 142
1O0ГР=1449" 100 ГР =14999
1OOГP=1666""" 100 ГР = 166
O0 RP=72" 100 ГР = 72
-177%% -17%
10O ШT= 91 100 ШТ = 91
ЮORP-52" 100 ГР = 52
РУБ РУБ
1OOГP =  142" 100 ГР = 142
100 Г= 97" 100 ГР = 97
1OOГP = 149*" 100 ГР = 14999
1OOГP=L66" 100 ГР = 16666
PУБ. РУБ.
1KГ= 127* 1 КГ = 127
1OORP=166"" 100 ГР = 16666
1KГ== 259" 1 КГ =25990
РУБ. РУБ.
РYБ. РУБ.
11KГ=255" 1 КГ = 25590
1OOГP=166" 100 ГР = 16666
1KГ

In [13]:
print('Levinstain distance: ', lev_dist/len(string_data))
print('Accurancy: ', match/len(string_data))

Levinstain distance:  0.3672954126545062
Accurancy:  0.16041440700153345


In [13]:
print('Levinstain distance: ', lev_dist/len(string_data))
print('Accurancy: ', match/len(string_data))

Levinstain distance:  0.3672954126545062
Accurancy:  0.16041440700153345
